In [6]:
# Path to exported model pipeline.config
PATH_TO_CFG = "./exported_models/run2/pipeline.config"

# Path to exported model checkpoint folder
PATH_TO_CKPT = "./exported_models/run2/checkpoint"

# Path to eval image
IMAGE_PATHS = ["./original1.jpg"]

In [2]:
import os
import pathlib
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

tf.get_logger().setLevel('ERROR')

import time

print(tf.__version__)
print("Available GPU devices: %s" % tf.test.gpu_device_name())

2.3.1
Available GPU devices: 


In [3]:
# Load our model
import time
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder



print("Loading model...", end='')
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    
    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print("Done! Took {} seconds".format(elapsed_time))

Loading model...Done! Took 0.0753927230834961 seconds


In [4]:
# Load label map data
category_index = label_map_util.create_category_index_from_labelmap("annotations/label_map.pbtxt", use_display_name=True)

In [19]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
# import warnings
# warnings.filterwarnings('ignore')

def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

for image_path in IMAGE_PATHS:
    print("Running inference for {}...".format(image_path), end='')
    
    image_np = load_image_into_numpy_array(image_path)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    
    detections = detect_fn(input_tensor)
    
    # Output is batch tensors
    # Take index [0] to remove batch dimension
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                 for key, value in detections.items()}
    detections['num_detections'] = num_detections
    
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    # Prune objects with high IoU with confident boxes
    selected_indices = tf.image.non_max_suppression(
        detections['detection_boxes'], 
        detections['detection_scores'], 
        max_output_size=200, 
        iou_threshold=0.3)
    selected_boxes = tf.gather(detections['detection_boxes'], selected_indices)
    selected_classes = tf.gather(detections['detection_classes'], selected_indices)
    selected_scores = tf.gather(detections['detection_scores'], selected_indices)
    
    sess = tf.compat.v1.Session()
    with sess.as_default():
        boxes = selected_boxes.numpy()
        classes = selected_classes.numpy()
        scores = selected_scores.numpy()
        
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        boxes,
        classes+label_id_offset,
        scores,
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.35,
        skip_labels=True,
        skip_scores=True,
        agnostic_mode=False)
    
    plt.figure()
    plt.imshow(image_np_with_detections)
    print(" Done!")

plt.savefig("inference.jpg")
print("File saved.")
#plt.show()

Running inference for ./original1.jpg... Done!
File saved.
